<a href="https://colab.research.google.com/github/Dhanushranga1/Diffusion-AIW3/blob/main/fine_tune_Stable_Diffusion_using_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

step1: installing required packages


In [1]:
!pip install diffusers==0.27.2 huggingface_hub==0.20.2 peft==0.4.0 accelerate==0.23.0 transformers==4.36.2 datasets pillow wandb --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.
gradio 5.31.0 requires huggingface-hub>=0.28.1, but you have huggingface-hub 0.20.2 which is incompatible.


step2: config and setup

In [2]:
import os

# Configuration
MODEL_NAME = "runwayml/stable-diffusion-v1-5"
INSTANCE_PROMPT = "a photo of sks car"  # change 'sks' to anything like 'neo', etc.
OUTPUT_DIR = "./lora_car_model"
RESOLUTION = 512

# Create folders
os.makedirs("./training_images", exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Now upload 5–10 images of cars to './training_images/' folder.")


✅ Now upload 5–10 images of cars to './training_images/' folder.


step3: data preparation consisting of resizing images

Resize and Clean Training Images to 512x512 for Stable Diffusion

In [4]:
from PIL import Image
import os

def prepare_training_images(input_dir="./training_images", size=512):
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print(" No images found in ./training_images. Please upload 3–10 car images.")
        return

    for file in image_files:
        img_path = os.path.join(input_dir, file)
        try:
            img = Image.open(img_path).convert("RGB")
            img = img.resize((size, size))
            img.save(img_path)
        except Exception as e:
            print(f" Could not process {file}: {e}")

    print(f" Resized and cleaned {len(image_files)} training images to {size}x{size}px.")


prepare_training_images()


 Resized and cleaned 9 training images to 512x512px.


step4: stable diffusion model and lora setup

Initialize Stable Diffusion v1.5 Pipeline and Apply LoRA (Low-Rank Adaptation) to UNet for Efficient Fine-Tuning

In [5]:
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model
import torch


pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False,
).to("cuda")


unet = pipe.unet

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.1,
    bias="none",
    task_type="TEXT_TO_IMAGE"
)
unet = get_peft_model(unet, lora_config)

print("Stable Diffusion + LoRA is ready.")


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datase

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Stable Diffusion + LoRA is ready.


step5: training lora model

In [10]:
vae = pipe.vae.cuda()
vae.eval()


AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

This step trains the UNet using LoRA adapters. We encoded images into latents using the VAE, added noise with the scheduler, and guided training using CLIP text embeddings.

Faced issues:
1. dtype mismatch (float vs float16)

2. backward pass errors

3.  needed to cast embeddings to half for UNet compatibility

In [19]:
from diffusers import DDIMScheduler
from transformers import CLIPTokenizer, CLIPTextModel
import torch.nn.functional as F
import torch

# Load tokenizer and text encoder
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14").cuda()

noise_scheduler = DDIMScheduler.from_pretrained(MODEL_NAME, subfolder="scheduler")
vae = pipe.vae.cuda()
vae.eval()

unet.train()

for step in range(MAX_TRAIN_STEPS):
    for batch in dataloader:
        batch = batch.to("cuda", dtype=torch.float16)

        text_input = tokenizer(INSTANCE_PROMPT, return_tensors="pt").input_ids.cuda()
        text_embeddings = text_encoder(text_input)[0].half()  # For float16 compatibility

        latents = vae.encode(batch).latent_dist.sample() * 0.18215  # Scale factor per SD docs

        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, 1000, (latents.size(0),), device=latents.device).long()
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        noise_pred = unet(
            noisy_latents,
            timesteps,
            encoder_hidden_states=text_embeddings
        ).sample

        loss = F.mse_loss(noise_pred, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        break

    if step % 50 == 0:
        print(f"Step {step}/{MAX_TRAIN_STEPS} | Loss: {loss.item():.4f}")

unet.save_pretrained(OUTPUT_DIR)
print(f" LoRA model saved to {OUTPUT_DIR}")


Step 0/400 | Loss: 0.0088
Step 50/400 | Loss: 0.0825
Step 100/400 | Loss: 0.6304
Step 150/400 | Loss: 0.0104
Step 200/400 | Loss: 0.0059
Step 250/400 | Loss: 0.0051
Step 300/400 | Loss: 0.1170
Step 350/400 | Loss: 0.0045
 LoRA model saved to ./lora_car_model


Load and Merge LoRA for Inference

We load the trained LoRA model into the base Stable Diffusion pipeline and optionally merge it with the base weights for faster inference. This step is needed before generating images using the fine-tuned model.


In [17]:
from diffusers import StableDiffusionPipeline
from peft import PeftModel
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
).to("cuda")

pipe.unet = PeftModel.from_pretrained(pipe.unet, "./lora_car_model")
pipe.unet = pipe.unet.merge_and_unload()  # Optional: merge LoRA with base weights

print("LoRA model loaded for inference.")


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

LoRA model loaded for inference.


In [18]:
prompt = "a red sports car on a highway, cinematic lighting, ultra detailed"
with torch.autocast("cuda"):
    image = pipe(prompt, num_inference_steps=30, guidance_scale=7.5).images[0]

image.show()
image.save("car_lora_sample.png")


  0%|          | 0/30 [00:00<?, ?it/s]

In [21]:
from PIL import Image as PILImage

img = PILImage.open("/content/car_lora_sample.png")
img.show()
